
<h1>Advanced Lane Finding Project</h1>

<b>by Y.Ivanov</b>

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


<h2><a href="https://review.udacity.com/#!/rubrics/1966/view">Rubric Points</a></h2>

Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  


<h2>Writeup / README</h2>

You're reading it!


<h2>Camera Calibration</h2>

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

Chess board can't be detected on images 1,4,5 and images 7 and 15 has been provded in wrong size:
<code>
findChessboardCorners ERROR on: ../camera_cal/calibration4.jpg
../camera_cal/calibration7.jpg (721, 1281)
findChessboardCorners ERROR on: ../camera_cal/calibration1.jpg
findChessboardCorners ERROR on: ../camera_cal/calibration5.jpg
../camera_cal/calibration15.jpg (721, 1281)
</code>

Here provided samples of original and undistoreed images 
<img src = "../output_images/01_cam_cal_01.png" alt="sample 01"> 
<img src = "../output_images/01_cam_cal_02.png" alt="sample 02"> 

<h2>Pipeline (test images)</h2>
<h3>1. A distortion-correcting </h3>
Undistortion function has been caled at line 182:
<code>
        image = cv2.undistort( image, self.mtx, self.dist )
</code>
<img src="../output_images/02_pipeline_01.png">

<h3>2. A color transformation </h3>
Performed by <b>hsv_select_v</b>
<code>
        image_sel = hsv_select_v( image, threshold=(self.v_treshold,255) )
</code>

<img src="../output_images/02_pipeline_02.png">

Threshold value depends from video and ligthning coditions and fine tuned manually.

<h3>3. Ceating a threshold binary image </h3>
All desribed in lection 7 Sobel transfromation has been applied one by one: 
<ul>
<li>Sobel absolute axis x:<br/>
<code>
        img_gradx = sobel_abs_threshold(image, orient='x')
</code>
<img src="../output_images/02_pipeline_03.png">
</li>
<li>Sobel absolute axis y:<br/>
<code>
        img_grady = sobel_abs_threshold(image, orient='y')
</code>
<img src="../output_images/02_pipeline_04.png">
</li>
<li>Sobel magnitude:<br/>
<code>
        img_mag_binary = sobel_mag_threshold(image)
</code>
<img src="../output_images/02_pipeline_05.png">
</li>
<li>Sobel direction<br/>
<code>
        img_dir_binary = sobel_dir_threshold(image)
</code>
<img src="../output_images/02_pipeline_06.png">
</li>
</ul>

Combine all these transformation to one image:
<code>
        img_combined[ (image==1) | ((img_gradx == 1) & (img_grady == 1)) | ((img_mag_binary == 1) & (img_dir_binary == 0)) ] = 1
</code>
<img src="../output_images/02_pipeline_comb.png">

<h3>4. A perspective transform</h3>
Source and destation for perspective trasfrom depends on concrete video and calculated manually:
<code>
        src = np.float32( [ [ 200, 690 ],
                        [ 640-coffs, top ],
                        [ 640+coffs, top ],
                        [ 1100, 690 ] ] )
</code>
Source:<br/>
<code>
            img_transformed = perspective_transform( img_combined, 
                                                top = self.pers_trans_top,
                                                coffs = self.pers_trans_coffs,
                                                verbose=self.verbose )
</code>
<img src="../output_images/02_pipeline_07.png">
Destination:<br/>
<code>
        dst = np.float32([[220, 720], 
                      [220, 0],
                      [1070, 0],
                      [1070, 720]])
</code>
<img src="../output_images/02_pipeline_08.png">

<h3>5. Identifing lane-line pixels and fit their positions with a polynomial</h3>
In case if polunomial is not yet available lane pixels detected by sliding windows across peaks of hystogram. In other case search around poly strataegy has been applied: 
<code>
        if len( self.left_fit ) == 0 and len( self.right_fit ) == 0 :            
            # Find our lane pixels first
            new_leftx, new_lefty, new_rightx, new_righty, out_img = find_lane_pixels( img_transformed )
        else:
            new_leftx, new_lefty, new_rightx, new_righty, out_img = self.search_around_poly( img_transformed )
</code>

Polynomials calculated base on x and y coordinates:
<code>
        new_left_fit = np.polyfit(new_lefty,  new_leftx,  2)
        new_right_fit = np.polyfit(new_righty, new_rightx, 2)
</code>

Also additinal checks has been performed for detecting edges and error cases: lines 260-299
<img src="../output_images/02_pipeline_09.png">

<h3>6. Calculated the radius of curvature of the position of the vehicle</h3>
Lanes curvature and offset calculation based on code from lesson 8
<code>
    def calculate_curvature_and_offset(self, leftx, lefty, rightx, righty ) :
</code>
lines 35-67

<h3>7. Result image</h3>

<img src="../output_images/02_pipeline_10.png">

<h2>Pipeline (video)</h2>
Preprocessed video available here:
<ul>
<li> <a href="../output_videos/01_project_video.mp4"> Project video </a> </li>
<li> <a href="../output_videos/02_challenge_video.mp4"> Challenge video </a> </li>
<li> On harder challenge video lanes <font color = "red">can't be</font> detected. Plase take a <a href="../output_videos/03_harder_challenge_video.mp4"> look </a> </li>
</ul>

<h2>Discussion</h2>

<h3>Problems</h3>
<ul>
    <li>Some of parameters can be only hadrcoded and should be tuned (choosed) manually</li>
    <li>Manual adjustment of paramets consume a lot of time</li>
    <li>Hard to investigate pipeline errors especially when memory mechanism has been introduced.</li> 
</ul>
<h3>Possible improvenets</h3>
<ul>
    <li>Implement some set of tools for autmate coosing of parameters like a RIO size, filters threshhold</li>
    <li>More sofisticate memory and spartial filters can be impelemented</li>
    <li>Debugging technick can be improved</li>
</ul>